# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

## Imports

In [77]:
from random import choices, choice, random, randrange, shuffle
from dataclasses import dataclass
from copy import copy
from tqdm.auto import tqdm

import lab9_lib

## EA Strategy

In [78]:
POPULATION_SIZE = 300
OFFSPRING_SIZE = 500
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .1
GENOTYPE_SIZE = 1000
N_GENERATIONS = 1000

In [79]:
def EA_strategy(fitness):
    
    @dataclass
    class Individual:
        genotype: list[int]
        fitness: float


    def select_parent(pop):
        """
        Selects a parent from the population using tournament selection.

        Parameters:
            pop (list[Individual]): The population of individuals.

        Returns:
            Individual: The selected parent.

        """

        pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion


    def mutate(ind: Individual) -> Individual:
        """
        Mutates an individual by flipping a random bit in its genotype.

        Parameters:
            ind (Individual): The individual to mutate.

        Returns:
            Individual: The mutated individual.

        """

        offspring = copy(ind)
        pos = randrange(0, GENOTYPE_SIZE)

        offspring.genotype[pos] = 1 - offspring.genotype[pos]
        offspring.fitness = None

        return offspring


    def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
        """
        Performs one-cut crossover between two individuals.

        Parameters:
            ind1 (Individual): The first individual.
            ind2 (Individual): The second individual.

        Returns:
            Individual: The offspring produced by one-cut crossover.
        """
        cut_point = randrange(0, GENOTYPE_SIZE)
        offspring = Individual(
            fitness=None, genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
        )
        assert len(offspring.genotype) == GENOTYPE_SIZE
        return offspring

    population = [
        Individual(genotype=choices([0, 1], k=GENOTYPE_SIZE), fitness=None)
        for _ in range(POPULATION_SIZE)
    ]

    for ind in population:
        ind.fitness = fitness(ind.genotype)

    population
    
    with tqdm(total=N_GENERATIONS * OFFSPRING_SIZE) as pbar:
        for generation in range(N_GENERATIONS):
            offspring = list()

            for counter in range(OFFSPRING_SIZE):
                # Mutation
                if random() < MUTATION_PROBABILITY:
                    p = select_parent(population)
                    o = mutate(p)
                # Crossover
                else:
                    p1 = select_parent(population)
                    p2 = select_parent(population)
                    o = one_cut_xover(p1, p2)

                offspring.append(o)

                pbar.update(1)

            # Evaluate fitness of offspring
            for i in offspring:
                i.fitness = fitness(i.genotype)

            # Combine offspring with the existing population
            combined_population = population + offspring

            # Shuffle the combined population
            shuffle(combined_population)

            # Select top individuals to form the new population
            combined_population.sort(key=lambda i: i.fitness, reverse=True)
            population = combined_population[:POPULATION_SIZE]

            # Check if optimal solution is found
            if population[0].fitness == 1.0:
                break

    print(f"Best fitness: {population[0].fitness * 100}%")
    print(f"Fitness calls: {fitness.calls}")

In [80]:
for problem_size in [1, 2, 5, 10]:
    print(f"Problem size: {problem_size}")
    fitness = lab9_lib.make_problem(problem_size)
    EA_strategy(fitness)
    print()

Problem size: 1


  0%|          | 0/500000 [00:00<?, ?it/s]

Best fitness: 100.0%
Fitness calls: 173800

Problem size: 2


  0%|          | 0/500000 [00:00<?, ?it/s]

Best fitness: 92.2%
Fitness calls: 500300

Problem size: 5


  0%|          | 0/500000 [00:00<?, ?it/s]

Best fitness: 48.5%
Fitness calls: 500300

Problem size: 10


  0%|          | 0/500000 [00:00<?, ?it/s]

Best fitness: 37.8%
Fitness calls: 500300

